In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install langchain faiss-cpu sentence-transformers
!pip install transformers accelerate bitsandbytes


In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",  # Instruct fine-tuned model
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# تجهيز النموذج للتوليد
FastLanguageModel.for_inference(model)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-3-e66d6376f3a8>:23: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf_llm = HuggingFacePipeline(pipeline=qa_pipeline)


In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# إعداد الـ pipeline مع النموذج
qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=False  # استخدام Greedy decoding
)

# لفه بـ HuggingFacePipeline
hf_llm = HuggingFacePipeline(pipeline=qa_pipeline)


Device set to use cpu


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# تحميل نموذج embeddings متعدد اللغات
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


<ipython-input-5-332ade682cd4>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
import json
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings

# ✅ تحميل النصوص من ملف JSON
json_file_path = '/content/footballQA.json'  # ✅ ضع مسار ملف JSON الخاص بك هنا
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
texts = data['texts']  # ✅ استخراج قائمة النصوص

# ✅ تحويل النصوص إلى مستندات
documents = [Document(page_content=text) for text in texts]

# ✅ تقسيم المستندات إلى مقاطع صغيرة
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# ✅ تحميل نموذج التضمين embeddings (حسب استخدامك)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")  # أو أي نموذج آخر مناسب

# ✅ إنشاء قاعدة بيانات FAISS من المستندات المقسمة
db = FAISS.from_documents(docs, embedding_model)

print("✅ تم بناء قاعدة بيانات FAISS بنجاح!")


✅ تم بناء قاعدة بيانات FAISS بنجاح!


In [ ]:
retriever = db.as_retriever(
    search_type="mmr",    # Maximal Marginal Relevance
    search_kwargs={"k": 3}  # استرجاع أفضل 3 مستندات
)


In [ ]:
from langchain.chains import RetrievalQA

# إعداد RAG Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
query = " عدد اللاعبين في كل فريق؟"  # سؤال باللغة العربية

result = qa_chain.invoke({"query": query})

print("💡 الجواب:", result["result"])
print("📄 المصدر:", result["source_documents"])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


💡 الجواب: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

"How many players are on each team? Each football team consists of 11 players, including the goalkeeper.",

"ما هو وقت المباراة الرسمي؟ تمتد المباراة إلى 90 دقيقة تُقسم إلى شوطين، كل شوط مدته 45 دقيقة، مع إمكانية إضافة وقت بدل ضائع حسب تقدير الحكم.",

"What is high pressing? It’s a tactic where players apply intense pressure in the opponent’s half to quickly regain possession.",

Question:  عدد اللاعبين في كل فريق؟
Helpful Answer: Each football team consists of 11 players, including the goalkeeper.
📄 المصدر: [Document(id='cdf7da1d-301f-4eb1-9690-4408c69b47a0', metadata={}, page_content='"How many players are on each team? Each football team consists of 11 players, including the goalkeeper.",'), Document(id='fe005b49-ee23-4bec-86a3-6f806cdd6888', metadata={}, page_content='"ما هو وقت المباراة الرسمي؟ تمتد المباراة إ

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
import re

# دالة لتبسيط النص
def normalize_text(text):
    text = text.lower()  # تحويل إلى حروف صغيرة
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)  # إزالة التشكيل
    text = re.sub(r'[أإآ]', 'ا', text)  # توحيد الهمزات إلى "ا"
    text = re.sub(r'[هة’]$', '', text)  # إزالة "ه" أو "ة" أو "’" من نهاية الكلمة
    text = re.sub(r'^ال', '', text) # إزالة ال التعريف من بداية الكلمة
    return text


In [ ]:
! pip install gTTS
from gtts import gTTS
import tempfile
import os
import gradio as gr


In [ ]:
!pip install SpeechRecognition
! pip install gradio gTTS SpeechRecognition playsound

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=5173819501b1445dda2d06d704af8f160915f1fee99990928c7461caaf77ecf7
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [ ]:
!pip install -q sentence-transformers


In [ ]:
from sentence_transformers import CrossEncoder

# ✅ تحميل نموذج إعادة ترتيب النتائج
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank(query, docs):
    pairs = [[query, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    # إعادة الترتيب حسب الدرجات
    ranked_docs = sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)
    return [doc for _, doc in ranked_docs]


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [1]:
import gradio as gr
import torch

# ✅ دالة كشف اللغة
def detect_language(text):
    arabic_chars = sum(1 for c in text if '\u0600' <= c <= '\u06FF')
    english_chars = sum(1 for c in text if c.isascii() and c.isalpha())

    if arabic_chars > english_chars:
        return "arabic"
    else:
        return "english"

# ✅ قائمة الردود الاجتماعية (يمكنك الإضافة أو التعديل)
social_responses = {
    "arabic": {
        "كيف حالك": "أنا بخير! شكرًا لسؤالك  كيف يمكنني مساعدتك اليوم؟",
        "كيفك":"أنا بخير! شكرًا لسؤالك  كيف يمكنني مساعدتك اليوم؟",
        "كيف حال":"أنا بخير! شكرًا لسؤالك  كيف يمكنني مساعدتك اليوم؟",
        "وش اخبارك":"أنا بخير! شكرًا لسؤالك  كيف يمكنني مساعدتك اليوم؟",
        "اخبارك":"أنا بخير! شكرًا لسؤالك  كيف يمكنني مساعدتك اليوم؟",
        "من أنت": "أنا ذكي مساعدك الرياضي !",
        "مين أنت":"أنا ذكي مساعدك الرياضي !",
        "من انت":"أنا ذكي مساعدك الرياضي !",
        "مين أنت":"أنا ذكي مساعدك الرياضي !",
        "مرحبا": "أهلًا وسهلًا! كيف يمكنني مساعدتك؟",
        "اهلا": "أهلًا وسهلًا! كيف يمكنني مساعدتك؟",
        "أهلا":"أهلًا وسهلًا! كيف يمكنني مساعدتك؟",
        "هاي":"أهلًا وسهلًا! كيف يمكنني مساعدتك؟",
        "أهلا وسهلا":"أهلًا وسهلًا! كيف يمكنني مساعدتك؟",
        "السلام عليكم": "وعليكم السلام ورحمة الله وبركاته! كيف يمكنني خدمتك؟",
        "السلام عليكم ورحمة الله": "وعليكم السلام ورحمة الله وبركاته! كيف يمكنني خدمتك؟",
        "مع السلامة":"مع السلامة! أتمنى لك يومًا موفقًا",
        "إلى اللقاء": "إلى اللقاء! لا تتردد في العودة متى ما احتجتني",
        "مع سلامة" :"مع السلامة! أتمنى لك يومًا موفقًا",
        "باي": "باي! إن احتجت أي شيء، أنا هنا دائمًا",
        "وداعا":"وداعًا! أتمنى لك التوفيق في يومك"
    },
    "english": {
        "how are you": "I'm great! Thanks for asking  How can I help you today?",
        "how are you doing": "I'm great! Thanks for asking  How can I help you today?",
        "how’s it going": "I'm great! Thanks for asking  How can I help you today?",
        "what do you do": "I'm Zaki, your sports assistant!",
        "hello": "Hello! How can I assist you?",
        "hi": "Hi there! How can I help you today?",
        "greetings": "Hello! How can I assist you?",
        "hey": "Hello! How can I assist you?",
        "howdy": "Hello! How can I assist you?",
        "hi": "Hi there! How can I help you today?",
        "goodbye": "Goodbye! Have a great day.",
        "bye": "Bye! Let me know if you need anything else.",
        "see you": "See you later! ",
        "see you later": "See you later! ",
        "farewell": "Farewell! It was nice talking to you.",
        "take care": "Take care! ",
        "have a good one": "Have a good one! ",
        "have a good day": "Have a good day! ",
        "have a nice day": "Have a nice day! ",
        "have a great day": "Have a great day! "

    }
}

# ✅ إعداد البرومبت حسب اللغة + الردود الاجتماعية
def generate_answer_with_context(user_input):
    language = detect_language(user_input)
    normalized_input = normalize_text(user_input)

    # الردود الاجتماعية
    responses = social_responses.get(language, {})
    for keyword, reply in responses.items():
        if normalize_text(keyword) in normalized_input:
            return f"✅ الإجابة: {reply}"

    # تابع بقية الكود كالمعتاد (RAG وLLM)...

    # 1️⃣ كشف اللغة تلقائيًا
    language = detect_language(user_input)
    # ✅ الردود الاجتماعية
    responses = social_responses.get(language, {})
    for keyword, reply in responses.items():
        # تبسيط الكلمة الرئيسية قبل المقارنة
        normalized_keyword = normalize_text(keyword)
        if normalized_keyword in normalize_text(user_input):
            return f"✅ الإجابة: {reply}"

    # 2️⃣ استرجاع الوثائق من قاعدة المعرفة
    retrieved_docs = retriever.get_relevant_documents(user_input)

    # 3️⃣ إعادة ترتيب الوثائق بناءً على المعنى
    reranked_docs = rerank(user_input, retrieved_docs)

    # 4️⃣ تجهيز السياق
    if reranked_docs:
        context = "\n".join([doc.page_content for doc in reranked_docs])
    else:
        context = "لا توجد معلومات متوفرة."

    # 5️⃣ إعداد الـ prompt حسب اللغة
    if language == "arabic":
        system_prompt = (
        "أنت مساعد ذكي. أجب على السؤال التالي باستخدام اللغة العربية، "
        "واستند على المعلومات المتوفرة في السياق التالي. "
        "لا تعطي اكثر من المعلومة المطلوبة "
        "افهم المعنى العام للسؤال حتى لو تغيرت صيغته.\n\n"
        f"{context}"
    )



    else:
        system_prompt = (
            "You are a smart assistant. Answer the question in English, "
            "And based on the information available in the following context."
            "Do not give more information than is required."
            "If the information is not explicitly present in the context, say: Not enough information."
            "Do not generate a greeting, summary, or explanation. "
            "based on the context below. Understand the meaning even if the question is paraphrased.\n\n"
            f"{context}"
        )

    # 6️⃣ إعداد الرسائل
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]

    # 7️⃣ تجهيز الإدخال للنموذج
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    attention_mask = inputs != tokenizer.pad_token_id

    # 8️⃣ توليد الإجابة
    outputs = model.generate(
        input_ids=inputs,
        attention_mask=attention_mask,
        max_new_tokens=300,
        do_sample=False,
        temperature=0.0,
        top_p=0.1,
        repetition_penalty=1.1,
    )

    # 9️⃣ تحويل الناتج إلى نص
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # إزالة كل ما قبل رد النموذج (لضمان الجواب الصافي فقط)
    if user_input in answer:
        cleaned_answer = answer.split(user_input)[-1].strip()
    else:
        cleaned_answer = answer


    # 10️⃣ فلترة إضافية (اختياري)
    if len(cleaned_answer) < 15 or "لا أعلم" in cleaned_answer:
        return " لا توجد معلومات كافية.\n\n هل لديك سؤال آخر؟\n\n Do you have another question?"

    return f" الإجابة: {cleaned_answer}\n\n هل لديك سؤال آخر؟\n\n Do you have another question?"



def generate_answer_with_audio(user_input):
    # الحصول على الرد النصي كما هو في كودك الأصلي
    answer_text = generate_answer_with_context(user_input)

    # استخراج النص فقط من الرد
    text = answer_text.replace(" الإجابة: ", "").strip()

    # توليد الصوت باستخدام gTTS
    language = "ar" if detect_language(user_input) == "arabic" else "en"
    tts = gTTS(text=text, lang=language)

    # حفظ الصوت في ملف مؤقت
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(tmp_file.name)

    return answer_text, tmp_file.name

# ✅ واجهة Gradio مع الربط الجديد
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 ذكي مساعدك الرياضي\n\n### اكتب سؤالك وسيتم البحث في قاعدة البيانات الخاصة بك.")

    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="📝 اكتب سؤالك هنا:")
            submit_btn = gr.Button("🔎 بحث")

        with gr.Column():
            output_text = gr.Markdown("الإجابة ستظهر هنا...")
            output_audio = gr.Audio(label="🔊 استمع للإجابة", autoplay=True)

    submit_btn.click(fn=generate_answer_with_audio, inputs=user_input, outputs=[output_text, output_audio])


# ✅ تشغيل الواجهة
demo.launch(share=True)


ModuleNotFoundError: No module named 'gradio'